# Doc2Vecの紹介
本紹介は，ツールの歴史的背景や詳細な技術的・構成的情報は簡素化し，ツール利用者向けのチュートリアルを指向しています．ご了承ください．

## Doc2Vecとは
Word2Vecは，単語をベクトル空間にマッピングして活用する技術の一つです．言い換えれば，単語の意味をベクトル空間上の数値として表そうとした技術とも言えます．「各単語は周辺語に関連する」というアイディアで機械学習技術を応用しており，様々な文章による学習結果として類似性や関連性がある単語がベクトル空間上の近しい位置に配置されることになります．

## Doc2Vec入門

ここでは，自然言語処理ライブラリである``gensim``によるdoc2vecの実装を利用する例を取り上げます．まずgensimをインストールするところから始めます．

In [ ]:
# gensimのインストール
!pip install gensim

### モデルの学習や保存の実行
次のプログラム例は，Doc2Vecによる学習や，その結果として得られるモデルを処理する例です．別資料で紹介しているWord2Vecと類似する部分が少なくありませんので，そちらも参照してください．

まず，ここでは最初に学習データとなるドキュメント（``documents``）を作成しています．各ドキュメントはドキュメントごとの単語のシーケンスをTaggedDocumentでラッパーしたものになります．

In [ ]:
from gensim.test.utils import common_texts #お試し用の学習データをgensimが用意している．
from gensim.models.doc2vec import Doc2Vec, TaggedDocument #Doc2VecとTaggedDocumentをインポート

# お試し用の学習データであるcommon_textsの各要素をTaggedDocumentでラッパーします．
# 各ドキュメントにはタグとして，たとえば連番の数字を与えるものとします．
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
documents

Doc2Vec(sentences=...)コンストラクタの実行により学習が行なわれます．このときの引数の種類は数多くのものがあります．完全なものとしては https://radimrehurek.com/gensim/models/doc2vec.html を参照してください．

本例に示していない引数としては，たとえば，``dbow_words``は，値が1の場合はDBOWによりskip-gramライクに学習し，0の場合はより高速な学習をすることになります．

Doc2VecはWord2Vecと同様に多くの文献がありますので本資料では技術的な説明は割愛し，基本的な用例を紹介するに留めます．

In [ ]:
# Doc2Vecで学習している．引数に学習データを与えます．
# vector_sizeはベクトルの次元数を表します．
# windowは，予測したい語から周辺語までの最大距離，この数値が高いほど多くの周辺語を扱うようになる．
# min_countは，学習に用いる語を決めるための語の頻出度への閾値，この閾値以上に登場した語を学習に用いる．
# workersは，学習に用いるスレッド数，リソースがあれば，workerが多いほど早めに処理が終わる可能性がある．
# epochsは世代．学習対象データを何度繰り返して学習するか．
model = Doc2Vec(documents, vector_size=100, window=2, min_count=2, workers=4, epochs=20)

大規模なコーパス（学習用に整理した文書データ）であれば，学習に時間がかかりますので，学習結果となるモデルをmodel.saveにより保存します．一度保存してしまえば，下記プログラムのようにloadすればモデルを再利用できます．

In [ ]:
# モデルと保存と読込ができる．
model.save("doc2vec_model")
model = Doc2Vec.load("doc2vec_model")

次のプログラムで，学習済みモデルを利用して新たな文書のベクトルを予測することができます．このinver_vecotrメソッドでもalphaやmin_alpha，ephocsといった引数を与えることができ，学習率や世代の調整が行なえます．

In [ ]:
# 学習済みモデルに基づいて新たな文書のベクトルを予測する．
vector = model.infer_vector(["system", "response"])
vector

Word2Vecと同様に，次のようにして与えた文書と類似性の高い文書を取得できます．

In [ ]:
sims = model.dv.most_similar(vector, topn=len(model.dv))
sims

上記で得られた結果に対し，最初のpairの一つ目の値(5)をインデックスとして，documentsの5番目の要素を取り出すと，一番類似度の高かった文書がわかります．（とはいえ，本例ではそこまで高い類似度には見えませんが）

In [ ]:
documents[5]

## 実践

最新のGensim，Pythonにより，手軽に使える学習済みモデルが無いので省略します．下記の「手持ちの文書で学習」の要領でDoc2Vecに大規模コーパスを学習させることで実践可能です．

たとえば，大規模コーパスの例にWikipediaの日本語記事が挙げられますが，``wget "https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles.xml.bz2" -O "jawiki-latest-pages-articles.xml.bz2"``を実行することで取得できます．ただし，5GBほどあるのでダウンロードに時間がかかります．

このbz2ファイルの内容をテキスト化してくれるツールとして``WikiExtractor``というものが存在します． https://github.com/attardi/wikiextractor WikiExtractorで出力したテキストは，階層的なファイル構造において，各XMLファイルにページとその内容が整理されていますので，ここからページごとに語を抽出します．そして，ページ＝文書とみなしてDoc2Vecで学習させます．モデルが得られれば，上述してきたように文書間の類似度などを計算することが可能です．


## 手持ちの文書で学習
ここでは，手持ちの文章で学習する方法を学びます．今回は処理速度を考慮して既に紹介しているMeCab (fugashi)を用いた方法を紹介します．

In [ ]:
# UniDic辞書を用いるfugashiのインストール
!pip install fugashi[unidic]
# 辞書(500MB程度)のダウンロード
!python -m unidic download
# 長文テキストの解析をデモする文章例
!wget 'https://www.aozora.gr.jp/cards/000148/files/773_14560.html' -O 'bunko.html' #こころ（青空文庫）
#ストップワードとして公開されているテキストを取得
!wget 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt' -O 'stopword.txt'

以下のプログラムでは名詞に限定して学習をさせます．また，今回は句点と空白で分割します．周辺語を適切に学習させるためには，適切に一文一文を抽出させることが望ましいですが，この処理は文章に応じた処理になりますので，ここでは扱いません．

In [ ]:
from bs4 import BeautifulSoup #マークアップ言語による記述からデータを抽出するためのライブラリ
from collections import Counter
from fugashi import Tagger
import re # 正規表現のライブラリ
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#単なるtxtファイルに書かれた文章を扱う場合はBeautifulSoupは不要です．
#そのtxtファイルをopenして文字列を取り出すのみで十分でしょう．
soup = BeautifulSoup(open("bunko.html", encoding="shift_jis"))
text = soup.find("div", "main_text").text #main_text classのdivタグに本文がある．
#ストップワードの読み込み，改行区切りでの単語分割
stopwords = open("stopword.txt", 'r').read().split('\n')
#カウントする形態素の品詞指定
allowed_pos = ["名詞"]

#分割しても改行コードなどは残る．名詞を取り上げるとして今回は不問にする．
#今回亜h，このように分割した文を文書に見立てる
sentences = re.split('。| | ', text)

tagger = Tagger()
documents = []
i = 0
for sentence in sentences :
    words = []
    for word in tagger(sentence) :
        if word.feature.orthBase not in stopwords and word.feature.pos1 in allowed_pos:
              words.append(str(word.feature.orthBase))
    if words :
        documents.append(TaggedDocument(words, [i]))
        i += 1

# モデルの生成（学習データとなる単語数が少ないためepochsを100にしているが，データによっては過学習に注意する必要がある．）
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=4, epochs=100)
documents

In [ ]:
# 647番目の文書は['惚', '先生', '人間', '幸福']
model.dv.most_similar(model.dv[647], topn=10)

In [ ]:
# 類似の文書を出力
print(documents[379]) #['先生', '幸福', '人間']
print(documents[648]) #['先生', '幸福']

## 演習
1. 手持ちの文書に対し，Doc2Vecを利用し，任意の語に対して類語を出力させ，その是非を評価しなさい．
1. Doc2Vecのパラメタを変化させ，出力の違いを観察しなさい．
1. DocVecに与える学習データについて，GinZaやMeCab，ならびに扱う品詞を調整することで出力にどのような差異が生じるかを調べなさい．

### 参考文献
1. Radim Řehůřek, GENSIM, https://radimrehurek.com/gensim/
1. PyPI, gensim 4.3.2, https://pypi.org/project/gensim/
1. @attardi, wikiextractor, https://github.com/attardi/wikiextractor
1. @taku910, MeCab: Yet Another Part-of-Speech and Morphological Analyzer, https://taku910.github.io/mecab/
1. @polm, fugashi, https://github.com/polm/fugashi
1. PyPI, fugashi 1.3.0, https://pypi.org/project/fugashi/
1. PyPi, unidic 1.1.0, https://pypi.org/project/unidic/
1. 国立国語研究所, UniDic, https://clrd.ninjal.ac.jp/unidic/download.html
1. 青空文庫, 青空文庫, https://www.aozora.gr.jp/

